# Livro Simpy - Simulação Discreta

## Link de Referencia
* https://simpy.livrosimulacao.eng.br/
* https://simpy.readthedocs.io/en/latest/contents.html

## Exercício de prática 1

In [1]:
import random             # gerador de números aleatórios
import simpy              # biblioteca de simulação

def distributions(tipo):
    TEMPO_MEDIO_CHEGADAS = 1.0          # tempo médio entre chegadas sucessivas de clientes
    TEMPO_MEDIO_ATENDIMENTO = 0.5       # tempo médio de atendimento no servidor
    
    distributions = {
        'chegada_cliente': random.expovariate(1.0/TEMPO_MEDIO_CHEGADAS),
        'atendimento_cliente': random.expovariate(1.0/TEMPO_MEDIO_ATENDIMENTO)
    }
    return distributions[tipo]


def geraChegadas(env):
    # função que cria chegadas de entidades no sistema
    contaChegada = 0
    while True:
        yield env.timeout(distributions('chegada_cliente'))
        contaChegada += 1
        print(f"Cliente {contaChegada} chega em: {env.now}")
        env.process(atendimentoServidor(env, contaChegada, servidorRes))


def atendimentoServidor(env, nome, servidorResource):
    request = servidorResource.request()
    tempo_chegada = env.now
    
    yield request
    print(f'Servidor inicia o atendimento do Cliente {nome} as: {env.now}')
    print(f'Tempo de permanência do Cliente {nome} na fila: {env.now - tempo_chegada}')
    yield env.timeout(distributions('atendimento_cliente'))
    print(f'Servidor termina o atendimento do Cliente {nome} as: {env.now}, Fila atual: {len(servidorRes.queue)}')
    yield servidorResource.release(request)
        
random.seed(1000)         # semente do gerador de números aleatórios
env = simpy.Environment() # cria o environment do modelo
servidorRes = simpy.Resource(env, capacity=1)
env.process(geraChegadas(env)) # cria o processo de chegadas
env.run(until=25) # roda a simulação por 10 unidades de tempo

Cliente 1 chega em: 1.5021840823801902
Servidor inicia o atendimento do Cliente 1 as: 1.5021840823801902
Tempo de permanência do Cliente 1 na fila: 0.0
Cliente 2 chega em: 1.6065890590717269
Servidor termina o atendimento do Cliente 1 as: 1.884703071733432, Fila atual: 1
Servidor inicia o atendimento do Cliente 2 as: 1.884703071733432
Tempo de permanência do Cliente 2 na fila: 0.2781140126617052
Servidor termina o atendimento do Cliente 2 as: 2.111161820540629, Fila atual: 0
Cliente 3 chega em: 5.437449718273912
Servidor inicia o atendimento do Cliente 3 as: 5.437449718273912
Tempo de permanência do Cliente 3 na fila: 0.0
Servidor termina o atendimento do Cliente 3 as: 5.566616886332236, Fila atual: 0
Cliente 4 chega em: 6.108513946323223
Servidor inicia o atendimento do Cliente 4 as: 6.108513946323223
Tempo de permanência do Cliente 4 na fila: 0.0
Servidor termina o atendimento do Cliente 4 as: 6.442717888676174, Fila atual: 0
Cliente 5 chega em: 6.721121419589968
Servidor inicia o at

## Exercício de prática 2 - Lavanderia

In [2]:
import random
import simpy

contaClientes=0             # conta clientes que chegaram no sistema

def distributions(tipo):
    # função que armazena as distribuições utilizadas no modelo
    return {
        'chegadas': random.expovariate(1.0/5.0),
        'lavar': 20,
        'descarregar': random.uniform(1, 4),
        'carregar': 2,
        'secar': random.uniform(9, 12),
    }.get(tipo, 0.0)

def chegadaClientes(env, lavadoras, cestos, secadoras):
    # função que gera a chegada de clientes
    global contaClientes
    while True:
        yield env.timeout(distributions('chegadas'))
        contaPecas += 1
        print(f"Cliente {contaPecas} chega em: {env.now}")
        

    # chamada do processo de lavagem e secagem
        env.process(lavaSeca(env, contaClientes, lavadoras, cestos, secadoras))

def lavaSeca(env, cliente, lavadoras, cestos, secadoras):
    # função que processa a operação de cada cliente dentro da lavanderia
    
    
    # ocupa a lavadora
    req_lavadora = lavadoras.request()
    yield req_lavadora
    print(f'Lavadora inicia a lavagem do Cliente {cliente} as: {env.now}')
    yield env.timeout(distributions('lavar'))
    
    # antes de retirar da lavadora, pega um cesto
    req_cesto = cestos.request()
    yield req_cesto
    print(f'Cliente {cliente} ocupa o cesto as: {env.now}')
    yield env.timeout(distributions('descarregar'))
    

    # libera a lavadora, mas não o cesto
    yield lavadoras.release(req_lavadora)
    print(f'Cliente {cliente} desocupa a lavadora as: {env.now}')

    # ocupa a secadora antes de liberar o cesto
    req_secadora = secadoras.request()
    yield req_secadora
    print(f'Cliente {cliente} ocupa a secadora as: {env.now}')
    yield env.timeout(distributions('carregar'))
    
    # libera o cesto mas não a secadora
    yield cestos.release(req_cesto)
    print(f'Cliente {cliente} desocupa o cesto as: {env.now}')
    yield env.timeout(distributions('secar'))
    
    # pode liberar a secadora
    print(f'Cliente {cliente} iniciar o descarregamento as: {env.now}')
    yield env.timeout(distributions('descarregar'))	
    
    yield secadoras.release(req_secadora)
    print(f'Cliente {cliente} desocupa o secadora as: {env.now}')
    

random.seed(10)
env = simpy.Environment()
lavadoras = simpy.Resource(env, capacity=3)
cestos = simpy.Resource(env, capacity=2)
secadoras = simpy.Resource(env, capacity=1)
env.process(chegadaClientes(env, lavadoras, cestos, secadoras))

env.run(until=40)

UnboundLocalError: cannot access local variable 'contaPecas' where it is not associated with a value

# Lista de Exercícios 1 - Simulação Discreta

## Lavanderia

In [16]:
import random
import simpy
import statistics

contaClientes = 0             # conta clientes que chegaram no sistema
tempoTotalCliente = [] #done
tempoFilaClientes = [] #done
tempoLavadoras = [] #done
tempoCestos = [] #done
tempoSecadoras = [] #done

def distributions(tipo):
    # função que armazena as distribuições utilizadas no modelo
    return {
        'chegadas': random.expovariate(1.0/10.0),
        'lavar': 25,
        'descarregar': random.uniform(1, 4),
        'carregar': 2,
        'transportar': random.uniform(3,5),
        'secar': random.uniform(9, 12),
    }.get(tipo, 0.0)

def chegadaClientes(env, lavadoras, cestos, secadoras):
    # função que gera a chegada de clientes
    global contaClientes
    while True:
        yield env.timeout(distributions('chegadas'))
        contaClientes += 1
        print(f"Cliente {contaClientes} chega em: {env.now}")
        

    # chamada do processo de lavagem e secagem
        env.process(lavaSeca(env, contaClientes, lavadoras, cestos, secadoras))

def lavaSeca(env, cliente, lavadoras, cestos, secadoras):
    # função que processa a operação de cada cliente dentro da lavanderia
    inicio_geral = env.now

    inicio_fila = env.now

    
    # ocupa a lavadora
    req_lavadora = lavadoras.request()
    yield req_lavadora
    inicio_lavar = env.now
    print(f'Lavadora inicia a lavagem do Cliente {cliente} as: {env.now}')
    
    termino_fila = env.now - inicio_fila
    tempoFilaClientes.append(termino_fila)
    
    yield env.timeout(distributions('lavar'))
    termino_lavar = env.now - inicio_lavar
    tempoLavadoras.append(termino_lavar)
    
    # antes de retirar da lavadora, pega um cesto
    req_cesto = cestos.request()
    yield req_cesto
    inicio_cesto = env.now
    print(f'Cliente {cliente} ocupa o cesto as: {env.now}')
    print(f'Cliente {cliente} inicia o descarregamento da maquina as: {env.now}')
    yield env.timeout(distributions('descarregar'))
    

    # libera a lavadora, mas não o cesto
    yield lavadoras.release(req_lavadora)
    print(f'Cliente {cliente} desocupa a lavadora as: {env.now}')
    
    # cliente trasporta o cesto ate a maquina de secar
    print(f'Cliente inicia o transporte das roupas para a máquina de secar as: {env.now}')
    yield env.timeout(distributions('transportar'))
    print(f'Cliente terminar o transporte das roupas para a máquina de secar as: {env.now}')
    
    # ocupa a secadora antes de liberar o cesto
    req_secadora = secadoras.request()
    yield req_secadora
    inicio_secadoras = env.now
    print(f'Cliente {cliente} ocupa a secadora as: {env.now}')
    yield env.timeout(distributions('carregar'))
    
    # libera o cesto mas não a secadora
    yield cestos.release(req_cesto)
    termino_cesto = env.now - inicio_cesto
    tempoCestos.append(termino_cesto)
    print(f'Cliente {cliente} desocupa o cesto as: {env.now}')
    yield env.timeout(distributions('secar'))
    
    # pode liberar a secadora
    print(f'Cliente {cliente} iniciar o descarregamento da secadora as: {env.now}')
    yield env.timeout(distributions('descarregar'))	
    
    yield secadoras.release(req_secadora)
    termino_secadoras = env.now - inicio_secadoras
    tempoSecadoras.append(termino_secadoras)

    termino_geral = env.now - inicio_geral
    tempoTotalCliente.append(termino_geral)
    print(f'Cliente {cliente} desocupa o secadora as: {env.now}')
    

random.seed(10)
env = simpy.Environment()
lavadoras = simpy.Resource(env, capacity=7)
cestos = simpy.Resource(env, capacity=12)
secadoras = simpy.Resource(env, capacity=1)
env.process(chegadaClientes(env, lavadoras, cestos, secadoras))

env.run(until=600)

print("Fila de clientes ao final da simulação: lavadoras %i cestos %i secadoras %i" 
        %(len(lavadoras.queue), len(cestos.queue), len(secadoras.queue)))


print(f'Tempo total do cliente: mean: {statistics.mean(tempoTotalCliente)}, standart deviation: {statistics.stdev(tempoTotalCliente)}, min: {min(tempoTotalCliente)}, max: {max(tempoTotalCliente)}') 
print(f'Tempo total do cliente em fila: mean: {statistics.mean(tempoFilaClientes)}, standart deviation: {statistics.stdev(tempoFilaClientes)}, min: {min(tempoFilaClientes)}, max: {max(tempoFilaClientes)}')
print(f'Tempo total do cliente em lavadoras: mean: {statistics.mean(tempoLavadoras)}, standart deviation: {statistics.stdev(tempoLavadoras)}, min: {min(tempoLavadoras)}, max: {max(tempoLavadoras)}')
print(f'Tempo total do cliente em cestos: mean: {statistics.mean(tempoCestos)}, standart deviation: {statistics.stdev(tempoCestos)}, min: {min(tempoCestos)}, max: {max(tempoCestos)}')
print(f'Tempo total do cliente em secadoras: mean: {statistics.mean(tempoSecadoras)}, standart deviation: {statistics.stdev(tempoSecadoras)}, min: {min(tempoSecadoras)}, max: {max(tempoSecadoras)}')

Cliente 1 chega em: 8.472372498338585
Lavadora inicia a lavagem do Cliente 1 as: 8.472372498338585
Cliente 2 chega em: 25.256033109779743
Lavadora inicia a lavagem do Cliente 2 as: 25.256033109779743
Cliente 1 ocupa o cesto as: 33.472372498338586
Cliente 1 inicia o descarregamento da maquina as: 33.472372498338586
Cliente 1 desocupa a lavadora as: 36.45791145849564
Cliente inicia o transporte das roupas para a máquina de secar as: 36.45791145849564
Cliente terminar o transporte das roupas para a máquina de secar as: 40.68456509958906
Cliente 1 ocupa a secadora as: 40.68456509958906
Cliente 1 desocupa o cesto as: 42.68456509958906
Cliente 2 ocupa o cesto as: 50.25603310977974
Cliente 2 inicia o descarregamento da maquina as: 50.25603310977974
Cliente 2 desocupa a lavadora as: 51.45101563816029
Cliente inicia o transporte das roupas para a máquina de secar as: 51.45101563816029
Cliente 1 iniciar o descarregamento da secadora as: 53.33151165162627
Cliente terminar o transporte das roupas 

## Cliníca Médica

In [15]:
import random
import simpy


contaClientes = 0             # conta clientes que chegaram no sistema
tempoTotalCliente = [] #done
tempoFilaClientes = [] #done
tempoMedicos = []
tempoRecepcionista = []

def distributions(tipo):
    # função que armazena as distribuições utilizadas no modelo
    return {
        'chegadas': random.expovariate(1.0/3.0),
        'preenchimentos': random.expovariate(1.0/10.0),
        'atendimento': random.expovariate(1.0/20.0),
        'pagamentos': random.uniform(1,4)
    }.get(tipo, 0.0)

def chegadaClientes(env, medicos, recepcionistas):
    # função que gera a chegada de clientes
    global contaClientes
    while True:
        yield env.timeout(distributions('chegadas'))
        contaClientes += 1
        print(f"Cliente {contaClientes} chega em: {env.now}")
    
    # chamada do processo de atendimentos
        env.process(atendeConsulta(env, contaClientes, medicos, recepcionistas))

def atendeConsulta(env, cliente, medicos, recepcionistas):
    # função que processa a operação de cada cliente dentro da lavanderia
    inicio_geral = env.now

    inicio_fila = env.now
    
    # ocupa a recepcionista
    print("Fila de clientes ao atual: medicos %i recepcionistas %i" 
        %(len(medicos.queue), len(recepcionistas.queue)))
    
    req_recepcionista = recepcionistas.request()
    yield req_recepcionista

    termino_fila = env.now - inicio_fila
    tempoFilaClientes.append(termino_fila)

    inicio_recepcionista = env.now
    print(f'Recepcionista inicia preenchimentos do Cliente {cliente} as: {env.now}')
    yield env.timeout(distributions('preenchimentos'))
    print(f'Recepcionista termina preenchimentos do Cliente {cliente} as: {env.now}')
    yield recepcionistas.release(req_recepcionista)
    termino_repectionista = env.now - inicio_recepcionista
    
    # ocupa o medico
    medico_inicio = env.now
    req_medicos = medicos.request()
    yield req_medicos
    print(f'Medico inicia preenchimentos do Cliente {cliente} as: {env.now}')
    yield env.timeout(distributions('atendimento'))
    print(f'Medico termina preenchimentos do Cliente {cliente} as: {env.now}')
    yield medicos.release(req_medicos)
    medico_final = env.now - medico_inicio
    tempoMedicos.append(medico_final)

    #ocupa a recepcionista para o pagamento
    inicio_fila = env.now
    inicio_recepcionista = env.now
    req_recepcionista = recepcionistas.request()
    print(f'Recepcionista inicia pagamento do Cliente {cliente} as: {env.now}')
    yield env.timeout(distributions('preenchimentos'))
    print(f'Recepcionista termina pagamento do Cliente {cliente} as: {env.now}')
    yield recepcionistas.release(req_recepcionista)

    termino_fila = termino_fila + (env.now - inicio_fila)

    termino_repectionista = termino_repectionista + (env.now - inicio_recepcionista)
    tempoRecepcionista.append(termino_repectionista)

    termino_geral = env.now - inicio_geral
    tempoTotalCliente.append(termino_geral)
    
    

random.seed(10)
env = simpy.Environment()
#
medicos = simpy.Resource(env, capacity=3)
recepcionistas = simpy.Resource(env, capacity=2)

#mundo ideal :)
# medicos = simpy.Resource(env, capacity=8)
# recepcionistas = simpy.Resource(env, capacity=13)
env.process(chegadaClientes(env, medicos, recepcionistas))

env.run(until=600)

print("Fila de clientes ao final da simulação: medicos %i recepcionistas %i" 
        %(len(medicos.queue), len(recepcionistas.queue)))


print(f'Tempo total do cliente: mean: {statistics.mean(tempoTotalCliente)}, standart deviation: {statistics.stdev(tempoTotalCliente)}, min: {min(tempoTotalCliente)}, max: {max(tempoTotalCliente)}') 
print(f'Tempo total do cliente em fila mean: {statistics.mean(tempoFilaClientes)}, standart deviation: {statistics.stdev(tempoFilaClientes)}, min: {min(tempoFilaClientes)}, max: {max(tempoFilaClientes)}')
print(f'Tempo total do cliente em medicos mean: {statistics.mean(tempoMedicos)}, standart deviation: {statistics.stdev(tempoMedicos)}, min: {min(tempoMedicos)}, max: {max(tempoMedicos)}')
print(f'Tempo total do cliente em recepcionista mean: {statistics.mean(tempoRecepcionista)}, standart deviation: {statistics.stdev(tempoRecepcionista)}, min: {min(tempoRecepcionista)}, max: {max(tempoRecepcionista)}')

Cliente 1 chega em: 2.5417117495015757
Fila de clientes ao atual: medicos 0 recepcionistas 0
Recepcionista inicia preenchimentos do Cliente 1 as: 2.5417117495015757
Recepcionista termina preenchimentos do Cliente 1 as: 6.51330092601302
Medico inicia preenchimentos do Cliente 1 as: 6.51330092601302
Cliente 2 chega em: 7.576809932933924
Fila de clientes ao atual: medicos 0 recepcionistas 0
Recepcionista inicia preenchimentos do Cliente 2 as: 7.576809932933924
Cliente 3 chega em: 9.018698317265528
Fila de clientes ao atual: medicos 0 recepcionistas 0
Recepcionista inicia preenchimentos do Cliente 3 as: 9.018698317265528
Recepcionista termina preenchimentos do Cliente 3 as: 10.457084786877097
Medico inicia preenchimentos do Cliente 3 as: 10.457084786877097
Medico termina preenchimentos do Cliente 3 as: 18.00524718397107
Recepcionista inicia pagamento do Cliente 3 as: 18.00524718397107
Recepcionista termina preenchimentos do Cliente 2 as: 18.419358057242675
Medico inicia preenchimentos do C

## Montagem

In [21]:
import random
import simpy

tempoFuncionarioFix = []
tempoFuncionarioUsi = []
tempoFuncionarioUne = []
tempoAbastecimento = []
qtdPecasAbastecimento = []
qtdParafusosAbastecimento = []

def distributions(tipo):
    # função que armazena as distribuições utilizadas no modelo
    return {
        'fixacao': random.normalvariate((2/3), (1/12)),
        'usinagem': 3,
        'uniao': random.uniform(3.5, 4),
        'abastecimento': random.normalvariate(40,3),
        'parafusos': random.uniform(990,1010),
        'pecas': 60
    }.get(tipo, 0.0)



def montagem(env, funcionario_fix_usi, funcionario_une, pecas, parafusos):
    while True:
        
        inicio_funcionario_fix = env.now
        funcionario_fix_usi_req = funcionario_fix_usi.request()
        yield funcionario_fix_usi_req
        print(f'Funcionario inicia fixacao as: {env.now}')

        yield pecas.get(2)
        yield env.timeout(distributions('fixacao'))

        termino_funcionario_fix = env.now - inicio_funcionario_fix
        tempoFuncionarioFix.append(termino_funcionario_fix)
        print(f'Funcionario finaliza fixacao as : {env.now}')
        yield funcionario_fix_usi.release(funcionario_fix_usi_req)

        inicio_funcionario_usi = env.now

        yield funcionario_fix_usi_req
        print(f'Funcionario inicia usinagem as : {env.now}')
        yield env.timeout(distributions('usinagem'))
        print(f'Funcionario finaliza usinagem as : {env.now}')
        yield funcionario_fix_usi.release(funcionario_fix_usi_req)
        termino_funcionario_usi = env.now - inicio_funcionario_usi
        tempoFuncionarioUsi.append(termino_funcionario_usi)

        inicio_funcionario_une = env.now
        funcionario_une_req = funcionario_une.request()
        yield funcionario_une_req
        print(f'Funcionario inicia uniao as: {env.now}')

        yield parafusos.get(4)
        
        yield env.timeout(distributions('uniao'))
        print(f'Funcionario finaliza uniao as : {env.now}')
        yield funcionario_une.release(funcionario_une_req)
        termino_funcionario_une = env.now - inicio_funcionario_une
        tempoFuncionarioUne.append(termino_funcionario_une)

def estoque(env, pecas, parafusos):
    while True:
        print(f'Atual nível do estoque: parafusos {parafusos.level}  pecas {pecas.level}')
        print(f'Inicia processo de abastecimento as: {env.now}')
        inicio_abastecimento = env.now
        yield env.timeout(distributions('abastecimento'))
        print(f'Termina processo de abastecimento as: {env.now}')
        termino_abastecimento = env.now - inicio_abastecimento
        tempoAbastecimento.append(termino_abastecimento)

        qtd_pecas = distributions('pecas')
        qtd_parafusos = distributions('parafusos')
        qtdPecasAbastecimento.append(qtd_pecas)
        qtdParafusosAbastecimento.append(qtd_parafusos)
        yield parafusos.put(qtd_parafusos)
        yield pecas.put(qtd_pecas)
        print(f'O estoque foi reabastecido com: parafusos {qtd_parafusos}  pecas {qtd_pecas}')


    

random.seed(10)
env = simpy.Environment()
parafusos = simpy.Container(env, init=990, capacity=2000)
pecas = simpy.Container(env, init=60, capacity= 70)
funcionario_fix_usi = simpy.Resource(env, capacity=1)
funcionario_une = simpy.Resource(env, capacity=1)
env.process(montagem(env, funcionario_fix_usi, funcionario_une, pecas, parafusos))
env.process(estoque(env, parafusos=parafusos, pecas=pecas))

env.run(until=600)

print(f'Nível final do estoque: parafusos {parafusos.level}  pecas {pecas.level}')


print(f'Tempo total da fixacao: mean: {statistics.mean(tempoFuncionarioFix)}, standart deviation: {statistics.stdev(tempoFuncionarioFix)}, min: {min(tempoFuncionarioFix)}, max: {max(tempoFuncionarioFix)}') 
print(f'Tempo total da usinagem mean: {statistics.mean(tempoFuncionarioUsi)}, standart deviation: {statistics.stdev(tempoFuncionarioUsi)}, min: {min(tempoFuncionarioUsi)}, max: {max(tempoFuncionarioUsi)}')
print(f'Tempo total da uniao mean: {statistics.mean(tempoFuncionarioUne)}, standart deviation: {statistics.stdev(tempoFuncionarioUne)}, min: {min(tempoFuncionarioUne)}, max: {max(tempoFuncionarioUne)}')
print(f'Tempo total do abastecimento mean: {statistics.mean(tempoAbastecimento)}, standart deviation: {statistics.stdev(tempoAbastecimento)}, min: {min(tempoAbastecimento)}, max: {max(tempoAbastecimento)}')
print(f'Quantidade total de pecas mean: {statistics.mean(qtdPecasAbastecimento)}, standart deviation: {statistics.stdev(qtdPecasAbastecimento)}, min: {min(qtdPecasAbastecimento)}, max: {max(qtdPecasAbastecimento)}')
print(f'Quantidade total de parafusos mean: {statistics.mean(qtdParafusosAbastecimento)}, standart deviation: {statistics.stdev(qtdParafusosAbastecimento)}, min: {min(qtdParafusosAbastecimento)}, max: {max(qtdParafusosAbastecimento)}')

Atual nível do estoque: parafusos 990  pecas 60
Inicia processo de abastecimento as: 0
Funcionario inicia fixacao as: 0
Funcionario finaliza fixacao as : 0.6710623606946468
Funcionario inicia usinagem as : 0.6710623606946468
Funcionario finaliza usinagem as : 3.6710623606946466
Funcionario inicia uniao as: 3.6710623606946466
Funcionario finaliza uniao as : 7.17308993277385
Funcionario inicia fixacao as: 7.17308993277385
Funcionario finaliza fixacao as : 7.780914864810733
Funcionario inicia usinagem as : 7.780914864810733
Funcionario finaliza usinagem as : 10.780914864810732
Funcionario inicia uniao as: 10.780914864810732
Funcionario finaliza uniao as : 14.708477735197533
Funcionario inicia fixacao as: 14.708477735197533
Funcionario finaliza fixacao as : 15.391470430495668
Funcionario inicia usinagem as : 15.391470430495668
Funcionario finaliza usinagem as : 18.391470430495666
Funcionario inicia uniao as: 18.391470430495666
Funcionario finaliza uniao as : 22.119337859836186
Funcionario 

## Logistica

Observações: 
* A questão não define uma distribuição para a quantidade de caixas por caminhão, apenas o volume por encomenda.

    * Por esse motivo, defini uma distribuição para a quantidade de caixas por caminhão dada pela distribuição triangular com low: 5, high: 20 e mode: 12
* Não há forma de colocar vários funcionários simultaneamente para um mesmo recurso
* O que fazer quando o armazem chegar no limite?

In [14]:
import random
import simpy

contaCaminhao = 0
contaVan = 0

def distributions(tipo):
    # função que armazena as distribuições utilizadas no modelo
    return {
        'chegada': random.expovariate(1.0/15.0),
        'entrega': random.normalvariate(4.0, 1.0),
        'descarregar': 1/12,
        'carregar': 1/20,
        'volume': random.triangular(0.03,1.0,0.12),
        'caixas': int(random.triangular(5, 20, 12)),
    }.get(tipo, 0.0)



    
def descarregarCaminhao(armazem, qtd_caixas):
    for _ in range(qtd_caixas):
        volume = distributions('volume')
        if armazem.level + volume < 300:
            armazem.put(volume)
            armazem_list.append(volume)
        else:
            print(f'Armazem vazio: {armazem.level}')
            return False
    print(f'Nivel do armazem apos descarregar caminhao: {armazem.level}')
    return True
            
def carregarVan(armazem, qtd_caixas):
    for _ in range(qtd_caixas+1):
        volume = distributions('volume')
        if armazem.level - volume >= 0:
            armazem.get(volume)
            armazem_list.pop(0)
        else:
            print(f'Armazem vazio: {armazem.level}')
            return False
    print(f'Nivel do armazem apos carregar van: {armazem.level}')
    return True

def chegadaCaminhao(env, patio_caminhao, funcionario, armazem, armazem_list):
    global contaCaminhao
    while True:
        if len(patio_caminhao.queue) < 5:
            yield env.timeout(distributions('chegada'))
            contaCaminhao += 1
            print(f"Caminhao {contaCaminhao} chega em: {env.now}")
    
            # chamada do processo de atendimentos
            patio_caminhao_req = patio_caminhao.request()
            print(f"Iniciando descarga do caminhão: {env.now}")
            yield patio_caminhao_req
            qtd_caixas = distributions('caixas')
            
            if funcionario.count == 0:
                funcionario_req = funcionario.request()
                yield funcionario_req
                print(f"6 Funcionarios ocupados as: {env.now}")
                resultado = descarregarCaminhao(armazem, qtd_caixas) 
                yield env.timeout((distributions('descarregar')/6.0) * qtd_caixas)
                  
            else: 
                funcionario_req = funcionario.request()
                yield funcionario_req
                print(f"4 Funcionarios ocupados as: {env.now}")
                resultado  = descarregarCaminhao(armazem, qtd_caixas)
                yield env.timeout((distributions('descarregar')/4.0) * qtd_caixas)
                
            
            yield funcionario.release(funcionario_req)
            print(f'Liberando funcionarios as: {env.now}')
            if resultado:
                yield patio_caminhao.release(patio_caminhao_req)
            else:
                print(f'Armazem cheio, descarregamento parado até liberação de espaço')
                yield patio_caminhao.release(patio_caminhao_req)
            
        env.process(saidaVan(env, van, patio_van, funcionario, armazem, armazem_list))
            
def saidaVan(env, van, patio_van, funcionario, armazem, armazem_list):
    global contaVan
    while True:
        if armazem_list:
            van_req = van.request()
            yield van_req
            print(f'Iniciando carga da Van as {env.now}')
            contaVan += 1
            patio_van_req = patio_van.request()
            yield patio_van_req
            qtd_caixas = caixaVan(armazem_list)
            if funcionario.count == 0:
                funcionario_req = funcionario.request()
                yield funcionario_req
                print(f"6 Funcionarios ocupados as: {env.now}")
                resultado = carregarVan(armazem, qtd_caixas)   
                yield env.timeout((distributions('carregar')/6.0) * qtd_caixas)
                
            else: 
                funcionario_req = funcionario.request()
                yield funcionario_req
                print(f"2 Funcionarios ocupados as: {env.now}")
                resultado  = carregarVan(armazem, qtd_caixas)
                yield env.timeout((distributions('carregar')/2.0) * qtd_caixas)
                
            
            yield funcionario.release(funcionario_req)
            print(f'Liberando funcionarios as: {env.now}')
            if resultado:
                yield patio_van.release(patio_van_req)
                print(f'Van começa o trajeto as: {env.now}')
                yield env.timeout(distributions('entrega'))
                print(f'Van retorna as: {env.now}')
                yield van.release(van_req)

            else:
                print(f'Armazem vazio, carregamento parado até chegada de mais mercadorias')
                yield patio_van.release(patio_van_req)

def percursoVan(env, van, req):
    yield env.timeout(distributions('entrega'))
    print(f'Van retorna as: {env.now}')
    yield van.release(req)

def caixaVan(armazem_list):
    volume = []
    for item in armazem_list:
        if sum(volume) + item < 8:
            return len(volume)
        else:
            volume += item
    return len(volume)
    

random.seed(10)
env = simpy.Environment()
armazem_list = []
armazem = simpy.Container(env, init=0, capacity=300)

patio_caminhao = simpy.Resource(env, capacity=1)
caminhao = simpy.Resource(env, capacity=5)

van = simpy.Resource(env, capacity=4)
patio_van = simpy.Resource(env, capacity=1)

funcionario = simpy.Resource(env, capacity=6)
env.process(chegadaCaminhao(env, patio_caminhao, funcionario, armazem, armazem_list))

env.run(until=800)

print(f'Nível final do armazem {armazem.level}. Numero de caminhoes: {contaCaminhao}. Numero de vans: {contaVan}')

Caminhao 1 chega em: 12.708558747507878
Iniciando descarga do caminhão: 12.708558747507878
6 Funcionarios ocupados as: 12.708558747507878
Nivel do armazem apos descarregar caminhao: 3.32241244954624
Liberando funcionarios as: 12.847447636396767
Iniciando carga da Van as 12.847447636396767
6 Funcionarios ocupados as: 12.847447636396767
Nivel do armazem apos carregar van: 2.9589326413696755
Liberando funcionarios as: 12.847447636396767
Van começa o trajeto as: 12.847447636396767
Van retorna as: 16.51859464029433
Iniciando carga da Van as 16.51859464029433
6 Funcionarios ocupados as: 16.51859464029433
Nivel do armazem apos carregar van: 2.665478984838565
Liberando funcionarios as: 16.51859464029433
Van começa o trajeto as: 16.51859464029433
Caminhao 2 chega em: 18.619667544635565
Iniciando descarga do caminhão: 18.619667544635565
6 Funcionarios ocupados as: 18.619667544635565
Nivel do armazem apos descarregar caminhao: 7.0584683072323795
Liberando funcionarios as: 18.744667544635565
Inici

## Bar Expresso

In [1]:
import random
import simpy

# Variáveis globais para acumular os tempos e contadores

acumulador_tempo_atendimento = 0
acumulador_tempo_servir = 0
acumulador_tempo_consumo = 0
contagem_clientes = 0

def distribuitions(tipo):
    """ Returns a random value based on the given distribution type """
    return {
        'chegada': random.expovariate(1 / 4),
        'tempo_atendimento': max(random.normalvariate(0.7, 0.3), 0.1),  # Ensure positive times
        'tempo_consumo_bebida_base': max(random.normalvariate(3, 1), 0.1),
        'tempo_lavar_guardar_no_freezer': max(random.normalvariate(0.5, 0.1), 0.1),
    }.get(tipo, 0.0)

probabilidade_copos = [0.3, 0.45, 0.2, 0.05]
tempo_copo_no_freezer = 4
max_copos_na_pia = 6

def gerar_chegadas(env, cadeiras, funcionarios, copos, pia, copos_sujos):
    global contagem_clientes
    cliente_id = 1
    while True:
        yield env.timeout(distribuitions('chegada'))
        env.process(cliente(env, cliente_id, cadeiras, funcionarios, copos, pia, copos_sujos))
        cliente_id += 1
        contagem_clientes += 1

def cliente(env, cliente_id, cadeiras, funcionarios, copos, pia, copos_sujos):
    global acumulador_tempo_atendimento, acumulador_tempo_servir
    global acumulador_tempo_consumo
    
    # Cliente chega e tenta sentar

    yield env.process(sentar_cadeira(env, cadeiras, cliente_id))

    # Atendimento do cliente
    copos_necessarios = random.choices([1, 2, 3, 4], probabilidade_copos)[0]
    inicio_atendimento = env.now
    yield env.process(atendimento_cliente(env, funcionarios, cliente_id, copos, copos_necessarios))
    acumulador_tempo_atendimento += env.now - inicio_atendimento

    # Servir a bebida
    inicio_servir = env.now
    yield env.process(servir_copo(env, copos, copos_necessarios, cliente_id))
    acumulador_tempo_servir += env.now - inicio_servir

    # Consumo da bebida
    inicio_consumo = env.now
    yield env.process(consumir_bebida(env, cliente_id, copos_necessarios))
    acumulador_tempo_consumo += env.now - inicio_consumo

    # Lavar e guardar copos
    for _ in range(copos_necessarios):
        yield env.process(lavar_copo(env, copos, pia, copos_sujos))
def sentar_cadeira(env, cadeiras, cliente_id):
    cadeira = cadeiras.request()
    yield cadeira
    try:
        print(f"Cliente {cliente_id} senta na cadeira. Tempo: {env.now:.2f}")
    finally:
        cadeiras.release(cadeira)

def atendimento_cliente(env, funcionarios, cliente_id, copos, copos_necessarios):
    funcionario = funcionarios.request()
    yield funcionario
    try:
        tempo_atendimento = distribuitions('tempo_atendimento')
        yield env.timeout(tempo_atendimento)
        print(f"Cliente {cliente_id} consumirá {copos_necessarios} copos. Tempo: {env.now:.2f}")
    finally:
        funcionarios.release(funcionario)

def servir_copo(env, copos, copos_necessarios, cliente_id):
    for _ in range(copos_necessarios):
        copo = copos.request()
        yield copo
        try:
            tempo_servir = distribuitions('tempo_atendimento')  # Simulando o tempo para servir um copo
            yield env.timeout(tempo_servir)
            print(f"Copo servido ao cliente {cliente_id}. Tempo: {env.now:.2f}")
        finally:
            copos.release(copo)

def consumir_bebida(env, cliente_id, copos_necessarios):
    tempo_consumo_base = distribuitions('tempo_consumo_bebida_base')
    tempo_consumo_total = tempo_consumo_base * copos_necessarios
    yield env.timeout(tempo_consumo_total)
    print(f"Cliente {cliente_id} terminou de consumir. Tempo: {env.now:.2f}")

def lavar_copo(env, copos, pia, copos_sujos):
    # Espera até que haja espaço na pia se estiver cheia
    if copos_sujos.level >= max_copos_na_pia:
        print(f"Pia cheia. Cliente aguardando liberação de espaço. Tempo: {env.now:.2f}")
        yield copos_sujos.get(max_copos_na_pia)
    
    copo = copos.request()
    yield copo
    try:
        pia_recurso = pia.request()
        yield pia_recurso
        try:
            tempo_lavar_guardar = distribuitions('tempo_lavar_guardar_no_freezer')
            yield env.timeout(tempo_lavar_guardar)
            print(f"Copo lavado e guardado no freezer. Tempo: {env.now:.2f}")
            
            copos_sujos.put(1)  # Adiciona um copo sujo à pia

            yield env.timeout(tempo_copo_no_freezer)
            print(f"Copo disponível para uso novamente. Tempo: {env.now:.2f}")
            copos_sujos.get(1)  # Remove um copo sujo da pia
        finally:
            pia.release(pia_recurso)
    finally:
        copos.release(copo)

def imprimir_estatisticas():
    """ Imprime estatísticas ao final da simulação """
    global acumulador_tempo_atendimento, acumulador_tempo_servir
    global acumulador_tempo_consumo, contagem_clientes

    if contagem_clientes > 0:
        print(f"\nEstatísticas da Simulação:")
        print(f"Tempo médio de atendimento: {acumulador_tempo_atendimento / contagem_clientes:.2f}")
        print(f"Tempo médio para servir um copo: {acumulador_tempo_servir / contagem_clientes:.2f}")
        print(f"Tempo médio de consumo por cliente: {acumulador_tempo_consumo / contagem_clientes:.2f}")

random.seed(50)
env = simpy.Environment()

cadeiras = simpy.Resource(env, capacity=6)
funcionarios = simpy.Resource(env, capacity=2)
copos = simpy.Resource(env, capacity=20)
pia = simpy.Resource(env, capacity=6)
copos_sujos = simpy.Container(env, capacity=max_copos_na_pia, init=0)

env.process(gerar_chegadas(env, cadeiras, funcionarios, copos, pia, copos_sujos))
env.run(until=90)

# Imprime estatísticas após a simulação
imprimir_estatisticas()


Cliente 1 senta na cadeira. Tempo: 2.75
Cliente 2 senta na cadeira. Tempo: 3.11
Cliente 1 consumirá 3 copos. Tempo: 4.02
Copo servido ao cliente 1. Tempo: 4.27
Cliente 3 senta na cadeira. Tempo: 4.35
Cliente 2 consumirá 2 copos. Tempo: 4.37
Cliente 3 consumirá 2 copos. Tempo: 4.73
Copo servido ao cliente 3. Tempo: 4.93
Copo servido ao cliente 2. Tempo: 5.09
Copo servido ao cliente 1. Tempo: 5.17
Cliente 4 senta na cadeira. Tempo: 5.48
Copo servido ao cliente 1. Tempo: 5.67
Copo servido ao cliente 3. Tempo: 5.89
Copo servido ao cliente 2. Tempo: 6.41
Cliente 4 consumirá 1 copos. Tempo: 6.64
Copo servido ao cliente 4. Tempo: 6.96
Cliente 4 terminou de consumir. Tempo: 10.07
Cliente 3 terminou de consumir. Tempo: 10.23
Copo lavado e guardado no freezer. Tempo: 10.59
Copo lavado e guardado no freezer. Tempo: 10.66
Cliente 5 senta na cadeira. Tempo: 11.58
Cliente 6 senta na cadeira. Tempo: 12.25
Cliente 5 consumirá 1 copos. Tempo: 12.56
Cliente 1 terminou de consumir. Tempo: 12.65
Cliente 6